## Test Performance Particle filter

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from src.models.particle_filter import ParticleFilterMLP, ParticleFilterModel
from src.models.normal import NormalDegradationModel as NModel
import imageio
import matplotlib.pyplot as plt

from src.helpers.visualization import create_pf_prediciton_frame

In [2]:
data_name = "DS03"
perform_name = "SmHPC"
device = "cpu"  # or "cuda"

pf_dir_name = 'pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05'

experiment_dir = Path('experiments')/data_name
pf_dir = experiment_dir/ pf_dir_name/perform_name
states_dir = experiment_dir/'states'/ perform_name

## Load component (base) models

In [3]:
hi_df = pd.read_csv(experiment_dir/'hidata_dev.csv')
dev_units = hi_df['unit'].astype(int).unique().tolist()
dev_units

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
degmodels = []
for unit in dev_units:
	best_model = NModel()
	best_model.load_state_dict(
		torch.load(states_dir /f'unit_{unit}'/ "best_model.pt", map_location=device)
	)
	best_model.to(device)
	degmodels.append(best_model) 
print(f'Number of component models: {len(degmodels)}')

Number of component models: 9


## Load test data

In [5]:
hi_df = pd.read_csv(experiment_dir/'hidata_test.csv')
test_units = hi_df['unit'].astype(int).unique().tolist()
test_units

[10, 11, 12, 13, 14, 15]

## Prepare data

In [6]:
performs = {unit: hi_df[hi_df['unit']==unit][perform_name].values for unit in test_units} 
time = {unit: hi_df[hi_df['unit']==unit]['cycle'].values for unit in test_units}

test_data = {unit: torch.tensor(np.stack([t_data, s_data],axis=1),dtype=torch.float32).to(device) 
             for t_data, s_data, unit in zip(time.values(), performs.values(), test_units)}

## Load PF-net

In [7]:
net = ParticleFilterMLP(state_dim=NModel.state_dim(),hidden_dims=[128, 128, 64,32],activation=lambda: nn.LeakyReLU(0.05))

ckpt = torch.load(pf_dir/'checkpoint.pt', map_location=device)
net.load_state_dict(ckpt['model_state'])

net = net.to(device)


net.eval()   

ParticleFilterMLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.05)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.05)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): LeakyReLU(negative_slope=0.05)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): LeakyReLU(negative_slope=0.05)
    (8): Linear(in_features=32, out_features=16, bias=True)
  )
)

## Hyper Parameted

In [8]:
n_particles = 1800
multiply_scale = 0.01
start_idx = 1

## Run Particle Filter with (leraned net)

Save video

In [9]:
t_grid = np.linspace(0.1, 100, 80)
s_grid = np.linspace(0.0, 1.0, 60)
conf_level = 0.95

for unit in test_units:
    t_data = test_data[unit][:, 0]
    s_data = test_data[unit][:, 1]
    t_data_np = t_data.cpu().numpy()
    s_data_np = s_data.cpu().numpy()

    pf = ParticleFilterModel(
        base_models=degmodels,
        net=net,  
        n_particles=n_particles,
        multiply_scale=multiply_scale,
        name=perform_name,
    ).eval()

    frames = []
    for k in range(start_idx, len(t_data)):
        pf.step(
            t_obs=t_data[:k+1],
            s_obs=s_data[:k+1],
        )
        lower, mean, upper = pf.mixture.uncertainty_interval(s=torch.zeros(1), level=conf_level)
        # --- render frame ---
        fig, ax = plt.subplots(figsize=(10, 8))
  
        create_pf_prediciton_frame(
            ax,
            pf,
            t_grid,
            s_grid,
            t_data_np,
            s_data_np,
            pred_interval=(lower.item(), mean.item(), upper.item()),
            conf_level=conf_level,
            current_step = k,
        )
  
        # create frame
        fig.canvas.draw()
        frame = np.asarray(fig.canvas.renderer.buffer_rgba())
        plt.close(fig)
        frames.append(frame)
  
    ## Save video
    video_path = pf_dir / f"pf_test{unit}.mp4"

    with imageio.get_writer(video_path, fps=8, macro_block_size=1) as writer:
        for frame in frames:
            writer.append_data(frame)

    print(f"🎬 Video saved to {video_path}")


🎬 Video saved to experiments/DS03/pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05/SmHPC/pf_test10.mp4
🎬 Video saved to experiments/DS03/pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05/SmHPC/pf_test11.mp4
🎬 Video saved to experiments/DS03/pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05/SmHPC/pf_test12.mp4
🎬 Video saved to experiments/DS03/pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05/SmHPC/pf_test13.mp4
🎬 Video saved to experiments/DS03/pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05/SmHPC/pf_test14.mp4
🎬 Video saved to experiments/DS03/pf_perform_forgetlik1_fullprior_128x128x64x32leaky0.05/SmHPC/pf_test15.mp4
